In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmphji4csk6/pubring.gpg' created
gpg: /tmp/tmphji4csk6/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [5]:
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

··········


In [6]:
!mkdir -p MyDrive
!google-drive-ocamlfuse MyDrive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [7]:
% cd MyDrive


/content/MyDrive


In [0]:
!ls


The objective of this experiment is to learn how to do Data preprocessing.

We will be using district wise demographics, enrollments, school and teacher indicator data to predict whether the literacy rate is high / medium / low in each district.

Data preprocessing is an important step of solving every machine learning problem. Most of the datasets used with Machine Learning problems need to be processed / cleaned / transformed so that a Machine Learning algorithm can be trained on it.

1. Data Cleaning → In this step the primary focus is on
    -Handling missing data
    -Handling nosiy data
    -Detection and removal of outliers

2. Data Integration → This process is used when data is gathered from various data sources
and data are combined to form consistent data. This data after performing cleaning is used
for analysis.

3. Data Transformation → In this step we will convert the raw data into a specified for-
mat according to the need of the model we are building. There are many options used for
transforming the data as below:
    
    -Normalization
    
    -Aggregation
    
    -Generalization

4. Data Reduction → After data transformation and scaling the redundancy within the data
is removed and efficiently organizing the data is performed.

# **GROUP LAB 2**

We have four different files

Districtwise_Basicdata.csv
Districtwise_Enrollment_details_indicator.csv
Districtwise_SchoolData.csv
Districtwise_Teacher_indicator.csv
These files contain the neccesary data to solve the problem. Load all the files correctly, after observing the header level details, data records etc

In [0]:

##If you wish to change the data files 
  DIR = "colab_docs/CSV"
  basicData = DIR +"/Districtwise_Basicdata.csv"
  enrollmentData = DIR + "/Districtwise_Enrollment_details_indicator.csv"
  schoolData = DIR + "/Districtwise_SchoolData.csv"
  teacherData = DIR + "/Districtwise_Teacher_indicator.csv"

In [0]:
def change_to_num(s):
  s = s.strip().lower()
  if s == 'high':
    return int('2')
  if s == 'medium':
    return int('1')
  return int('0')

In [0]:

import pandas as pd
basicdata = pd.read_csv(basicData,header=1,converters={'overall_lit': change_to_num} )
enrollmentdata = pd.read_csv(enrollmentData,header=3)
schooldata = pd.read_csv(schoolData, header=3)
teacherdata = pd.read_csv(teacherData,header=3)




In [0]:
# 
# import os
# files = []
# for f in os.walk(DIR):
#     files.append(f)

# list = files[0][-1]
# list


Remove the unwanted columns, which are unlikely to contribute for the prediction of overall literacy category(Manual Feature Selection)

The data is present in different files. We need to integrate all of the data into a single data frame.

For example all the data of the Anantapur district (AP) should form a single row.
Recall your SQL: create or identify a unique identifier for each row in all the data frames and use it to combine the data.
One possibility -- by no means the best-- is (year, state code, district code) combination

In [0]:
SubSchooldata=schooldata[['distcd','schgovt1','schgovt2','schgovt3','schgovt4','schgovt5','schgovt6','schgovt7','schgovt9','schpvt1','schpvt2','schpvt3','schpvt4','schpvt5','schpvt6','schpvt7','schpvt9']]
SubEnrolldata = enrollmentdata[['distcd','Enr Govt1','Enr Govt2','Enr Govt3','Enr Govt4','Enr Govt5','Enr Govt6','Enr Govt7','Enr Govt9','Enr Pvt1','Enr Pvt2','Enr Pvt3','Enr Pvt4','Enr Pvt5','Enr Pvt6','Enr Pvt7','Enr Pvt9']]
SubBasicdata = basicdata[['distcd', 'totpopulation','overall_lit']]
subTeacherData = teacherdata[['distcd', 'tch_govt2', 'tch_govt3', 'tch_govt4', 'tch_govt5', 'tch_govt6', 'tch_govt7', 'tch_govt9', 'tch_pvt1', 'tch_pvt2', 'tch_pvt3', 'tch_pvt4', 'tch_pvt5', 'tch_pvt6', 'tch_pvt7', 'tch_pvt9']]

#Sort the data based on distcd
SubSchooldata.sort_values('distcd',ascending=True)
SubBasicdata.sort_values('distcd', ascending=True)
SubEnrolldata.sort_values('distcd', ascending=True)
subTeacherData.sort_values('distcd', ascending=True)

Mergedata = pd.concat([SubSchooldata,SubEnrolldata,SubBasicdata],axis=1,join='inner')


In [0]:
## drop duplicate columns caused due to inner join
Mergedata = Mergedata.loc[:,~Mergedata.columns.duplicated()]
#Mergedata

In [0]:
## to drop all na columns
# Mergedata.dropna(inplace=True)
# Mergedata

Carry out the following steps to clean the data:

Overall_lit is our label. Without it we cannot meaningfully train.

So we can delete the rows with missing values in that column.
Bonus What else can we do with it?
Consider replacing the missing values in any other column

possibly with mean/median/mode.


In [0]:
Mergedata.isna().any()

In [0]:
## all columns that contain null values
nullcolumn = []
nullcolumn = Mergedata.columns[Mergedata.isnull().any()].tolist()
#nullcolumn

In [0]:
## number of nulls each column contains
na_col = []
list_na =[]
na_col = Mergedata.isnull().sum().tolist()
list_na = zip(nullcolumn,na_col)
#list_na

In [0]:
# for 
# Mergedata["schgovt4"].fillna(Mergedata["schgovt4"].mean())

In [0]:
for t in list_na:
  print(t[0])

In [0]:
## each null cells of the columns fill with mean of that particular column
for t in list_na:
  Mergedata[t[0]]=Mergedata[t[0]].fillna(Mergedata[t[0]].mean())
  


In [30]:
## to check that no null  columns are left
Mergedata.columns[Mergedata.isna().any()].tolist()

[]

In [141]:
Mergedata

,distcd,ac_year,schgovt1,schgovt2,schgovt3,schgovt4,schgovt5,schgovt6,schgovt7,schgovt9,...,Enr Pvt1,Enr Pvt2,Enr Pvt3,Enr Pvt4,Enr Pvt5,Enr Pvt6,Enr Pvt7,Enr Pvt9,totpopulation,overall_lit
0,3501,2012-13,61,26,19,0.000000,5,14.0,0.0,0.00000,...,2058.000000,1994.000000,5408.00000,0,0.000000,1153.000000,0.000000,0.00000,237586.0,2
1,3503,2012-13,104,29,5,3.000000,8,10.0,0.0,0.00000,...,779.000000,295.000000,19126.76003,0,0.000000,225.000000,0.000000,0.00000,105539.0,2
2,3502,2012-13,29,11,3,0.000000,6,7.0,0.0,0.00000,...,16.000000,29643.218608,19126.76003,0,8521.866969,9806.485261,5570.038549,2.59486,36819.0,2
3,2801,2012-13,3074,440,4,0.000000,1,97.0,430.0,0.00000,...,57987.000000,45282.000000,0.00000,0,0.000000,8169.000000,34569.000000,0.00000,2737738.0,0
4,2822,2012-13,2864,654,23,13.000000,495,1.0,50.0,0.00000,...,78173.000000,50979.000000,1704.00000,0,41617.000000,507.000000,6684.000000,0.00000,4083315.0,0
5,2823,2012-13,4234,528,21,0.000000,582,0.0,2.0,0.00000,...,89882.000000,38543.000000,3905.00000,0,50081.000000,484.000000,3557.000000,0.00000,4170468.0,1
6,2820,2012-13,2845,302,19,0.000000,361,0.0,7.0,1.00000,...,83877.000000,41090.000000,1321.00000,0,49146.000000,1419.000000,11333.000000,0.00000,2884524.0,1
7,2814,2012-13,3176,319,10,1.000000,0,73.0,532.0,0.00000,...,101920.000000,80477.000000,529.00000,0,304.000000,12728.000000,57451.000000,0.00000,5151549.0,1
8,2817,2012-13,2706,369,22,0.000000,379,0.0,0.0,0.00000,...,75736.000000,32198.000000,5001.00000,0,57450.000000,4101.000000,2668.000000,0.00000,4889230.0,1
9,2805,2012-13,621,10,2,5.000000,183,0.0,8.0,0.00000,...,233463.000000,64291.000000,36175.00000,0,145073.000000,4741.000000,8930.000000,0.00000,4010238.0,2


Use the functions below to adjust the outliers

smooth_out

takes a dataframe as input,
calculates the mean ( μ ) and standard deviation ( σ ) of every column
and replaces outliers with boundary values.
Outliers are those beyond  2σ  from  μ

In [0]:
import numpy as np

In [0]:
def clip_clamp(x, mean, sd):
  if x < mean - 2 * sd :
    return mean - 2 * sd
  elif x > mean + 2 * sd :
    return mean + 2 * sd
  else :
    return x
      

In [0]:
# Function to smooth the data
def smooth_out(Total_data):
    for i in Total_data.columns:
        if (isinstance(i, str)== False):
          mean = np.mean(Total_data[i].values, axis=0)
          sd = np.std(Total_data[i].values, axis=0)
          corrected = np.array([clip_clamp(x, mean, sd) for x in Total_data[i].values])
          Total_data[i] = pd.Series(corrected, index=Total_data[i].index)
    return Total_data

In [0]:
# np.array([1,2,3,4,5])
# total = pd.Series(np.array([1,2,3,4,5]),indes)

In [0]:
pd.Series?


In [34]:
smooth_out(Mergedata)

,distcd,schgovt1,schgovt2,schgovt3,schgovt4,schgovt5,schgovt6,schgovt7,schgovt9,schpvt1,...,Enr Pvt1,Enr Pvt2,Enr Pvt3,Enr Pvt4,Enr Pvt5,Enr Pvt6,Enr Pvt7,Enr Pvt9,totpopulation,overall_lit
0,3501,61,26,19,0.000000,5,14.0,0.0,0.00000,66,...,2058.000000,1994.000000,5408.00000,0,0.000000,1153.000000,0.000000,0.00000,237586.0,2
1,3503,104,29,5,3.000000,8,10.0,0.0,0.00000,18,...,779.000000,295.000000,19126.76003,0,0.000000,225.000000,0.000000,0.00000,105539.0,2
2,3502,29,11,3,0.000000,6,7.0,0.0,0.00000,2,...,16.000000,29643.218608,19126.76003,0,8521.866969,9806.485261,5570.038549,2.59486,36819.0,2
3,2801,3074,440,4,0.000000,1,97.0,430.0,0.00000,275,...,57987.000000,45282.000000,0.00000,0,0.000000,8169.000000,34569.000000,0.00000,2737738.0,0
4,2822,2864,654,23,13.000000,495,1.0,50.0,0.00000,356,...,78173.000000,50979.000000,1704.00000,0,41617.000000,507.000000,6684.000000,0.00000,4083315.0,0
5,2823,4234,528,21,0.000000,582,0.0,2.0,0.00000,440,...,89882.000000,38543.000000,3905.00000,0,50081.000000,484.000000,3557.000000,0.00000,4170468.0,1
6,2820,2845,302,19,0.000000,361,0.0,7.0,1.00000,409,...,83877.000000,41090.000000,1321.00000,0,49146.000000,1419.000000,11333.000000,0.00000,2884524.0,1
7,2814,3176,319,10,1.000000,0,73.0,532.0,0.00000,556,...,101920.000000,80477.000000,529.00000,0,304.000000,12728.000000,57451.000000,0.00000,5151549.0,1
8,2817,2706,369,22,0.000000,379,0.0,0.0,0.00000,424,...,75736.000000,32198.000000,5001.00000,0,57450.000000,4101.000000,2668.000000,0.00000,4889230.0,1
9,2805,621,10,2,5.000000,183,0.0,8.0,0.00000,891,...,233463.000000,64291.000000,36175.00000,0,145073.000000,4741.000000,8930.000000,0.00000,4010238.0,2



Use the function below (corr_features) to identify uncorrelated features and remove the remaining (dependent) features

corr_features takes pandas dataframe, columns in the dataframe and bar (correlation co-efficient)


In [0]:
# Function to find uncorrelated features
def corr_features(df, cols, bar=0.9):
  correlated = set()
  for pos, aCol in enumerate(cols[:-1]):
    if aCol in correlated: ## aCol has already been found dependent
      continue
    else: 
      for bCol in cols[pos + 1:]:
        score = df[aCol].corr(df[bCol])
        if -bar < score < bar:
          pass ## The correlation coefficient is within set limits
        else:
          correlated.add(bCol)
  return (set(cols) - correlated)

In [36]:
features = [x for x in corr_features(Mergedata, Mergedata.columns)]
uncorrelated = Mergedata[features]
uncorrelated


,schpvt2,Enr Pvt5,Enr Govt3,schgovt2,schpvt7,schgovt9,schgovt4,Enr Govt9,schpvt9,schgovt7,...,schpvt4,Enr Pvt2,Enr Govt2,schpvt6,distcd,Enr Govt1,schpvt1,overall_lit,Enr Pvt7,Enr Govt5
0,9,0.000000,10620,26,0.0,0.00000,0.000000,0.0,0.0,0.0,...,0.000000,1994.000000,3359.0,4,3501,3232,66,2,0.000000,1027
1,3,0.000000,1162,29,0.0,0.00000,3.000000,0.0,0.0,0.0,...,0.000000,295.000000,3808.0,1,3503,3996,18,2,0.000000,1397
2,0,8521.866969,357,11,0.0,0.00000,0.000000,0.0,0.0,0.0,...,0.000000,29643.218608,886.0,0,3502,1510,2,2,5570.038549,651
3,288,0.000000,1266,440,263.0,0.00000,0.000000,0.0,0.0,430.0,...,0.000000,45282.000000,53374.0,27,2801,135664,275,0,34569.000000,226
4,327,41617.000000,6867,654,54.0,0.00000,13.000000,0.0,0.0,50.0,...,0.000000,50979.000000,71562.0,2,2822,145256,356,0,6684.000000,92640
5,269,50081.000000,6305,528,36.0,0.00000,0.000000,0.0,0.0,2.0,...,0.000000,38543.000000,42118.0,2,2823,159102,440,1,3557.000000,97326
6,227,49146.000000,4710,302,71.0,1.00000,0.000000,0.0,0.0,7.0,...,0.000000,41090.000000,23931.0,6,2820,114284,409,1,11333.000000,52586
7,578,304.000000,2812,319,434.0,0.00000,1.000000,0.0,0.0,532.0,...,0.000000,80477.000000,43096.0,25,2814,183269,556,1,57451.000000,0
8,165,57450.000000,6898,369,15.0,0.00000,0.000000,0.0,0.0,0.0,...,0.000000,32198.000000,44585.0,11,2817,150932,424,1,2668.000000,82112
9,337,145073.000000,1687,10,86.0,0.00000,5.000000,0.0,0.0,8.0,...,0.000000,64291.000000,1587.0,14,2805,71152,891,2,8930.000000,29778


Perform Mean Correction and Standard Scaling feature/column wise on the data

In [37]:
import sklearn
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
transformed = scaler.fit_transform(uncorrelated)

transformed

array([[-0.67719538, -0.4260897 ,  1.00336882, ...,  1.05990127,
        -0.25667988, -0.26964654],
       [-0.71085684, -0.4260897 , -0.21374395, ...,  1.05990127,
        -0.25667988, -0.2607211 ],
       [-0.72768758,  0.        , -0.31733623, ...,  1.05990127,
         0.        , -0.27871672],
       ...,
       [-0.49766759, -0.16609192,  0.10514011, ...,  1.05990127,
        -0.14156643,  1.07363243],
       [-0.36302174, -0.38944001, -0.2991915 , ...,  1.05990127,
        -0.1569579 ,  8.57223367],
       [-0.63792368, -0.4260897 , -0.28413523, ..., -1.48232569,
        -0.24723302,  3.2839095 ]])

Split the data into train and test sets

In [0]:
train_set = {}
test_set = {}

n_train = 0
n_test = 0


split = int(0.95*len(transformed))
train_set = transformed[:split]

test_set = transformed[split:-1]
n_train = len(train_set)
n_test = len(test_set)

In [0]:
train_label = Mergedata['overall_lit'].values[:split]
test_label = Mergedata['overall_lit'].values[split:]


Apply k-NN and Decision Tree classifiers on the preprocessed data and figure out which classifier gives the best result.

In [0]:
from sklearn.neighbors import KNeighborsClassifier

def KNN(k):
  neighbour = KNeighborsClassifier(k).fit(train_set,train_label)
  print(neighbour.predict(test_set))

In [49]:
KNN(3)

[1 1 1 0 0 0 1 0 1 0 1 2 1 1 2 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 2 2 2 2
 2 2 2 1 2 2 2 2 1 2 1 2 1 2 2 2 2 0 1 2 0 2 2 2 2 2 0 2 2]


In [0]:
def accurate():
  Accuracy = list(zip(neighbour.predict(test_set),test_label))
  x = len(Accuracy)
  correct = [l for l,m in Accuracy if l==m]
  y = len(correct)

  print(x,y)
  z = (y/x)*100
  print(z)


In [54]:
accurate()

66 54
81.81818181818183


In [55]:
KNN(5)

[1 1 1 0 0 0 1 0 1 0 1 2 1 1 2 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 2 2 2 2
 2 2 2 1 2 2 2 2 1 2 1 2 1 1 2 1 1 1 1 2 0 2 1 2 2 2 1 2 2]


In [56]:
accurate()

66 54
81.81818181818183


In [67]:
KNN(31)

[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 2 2 2
 2 2 2 1 2 2 2 2 1 2 2 1 1 1 2 2 2 1 2 2 1 1 1 1 1 1 1 2 1]


In [68]:
accurate()

66 54
81.81818181818183
